# Getting Started with Hugging Face Models

This notebook will guide you through:
1. Setting up your environment for Hugging Face
2. Pulling a model from the Hugging Face Hub
3. Loading and using the model for inference
4. Fine-tuning a model for your specific needs (basic example)
5. Saving and sharing your model

Let's start with the basics and explore the Hugging Face ecosystem!

## 1. Setting Up Your Environment

First, we need to install the necessary packages. The main package we'll use is `transformers`, which is Hugging Face's primary library for working with pre-trained models.

In [ ]:
# Install the required packages
!pip install transformers datasets torch accelerate evaluate

# Check the version to make sure installation was successful
import transformers
print(f"Transformers version: {transformers.__version__}")

### Setting up Hugging Face Authentication (Optional)

If you want to access gated models or upload your own models, you'll need to authenticate with Hugging Face. This step is optional for public models.

In [ ]:
from huggingface_hub import login

# Run this cell and enter your token when prompted
# You can get your token from https://huggingface.co/settings/tokens
login()

## 2. Pulling a Model from Hugging Face Hub

Hugging Face offers thousands of models across various tasks. Let's start with a simple text classification model.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# This is a sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print(f"Model loaded: {model_name}")
print(f"Model size: {model.num_parameters():,} parameters")

### Understanding Model and Tokenizer

- **Model**: The neural network architecture loaded with pre-trained weights
- **Tokenizer**: Converts raw text into tokens that the model can understand

Let's explore what the model can do.

In [ ]:
# Check the model's configuration
print("Model configuration:")
print(model.config)

## 3. Using the Model for Inference

Now let's use our model to analyze some text.

In [ ]:
import torch
import numpy as np

# Define a function to get sentiment predictions
def analyze_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    # Get predictions from the model
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)
    
    # Convert to probabilities with softmax
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Get the most likely class
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    # Map class index to label
    sentiment = "positive" if predicted_class == 1 else "negative"
    confidence = probabilities[0][predicted_class].item()
    
    return sentiment, confidence

In [ ]:
# Try it with some example texts
examples = [
    "I love this product! It's amazing and works perfectly.",
    "The service was terrible and the staff was rude.",
    "The movie was okay, not great but not terrible either."
]

for text in examples:
    sentiment, confidence = analyze_sentiment(text)
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment} (Confidence: {confidence:.4f})")
    print("---")

### Try a Different Model: Text Generation

Let's also try a text generation model to see how easy it is to switch between different types of models.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Using a small GPT-2 model for text generation
gen_model_name = "distilgpt2"

# Create a text generation pipeline
generator = pipeline('text-generation', model=gen_model_name)

# Generate text
prompt = "In a world where AI and humans work together, "
result = generator(prompt, max_length=50, num_return_sequences=1)

print("Generated text:")
print(result[0]['generated_text'])

## 4. Fine-tuning a Model for Your Specific Task

One of the most powerful features of Hugging Face is the ability to fine-tune pre-trained models on your own data. Let's see how to fine-tune a text classification model on a simple dataset.

In [ ]:
from datasets import load_dataset
from transformers import Trainer, TrainingArguments

# Load a small dataset for demonstration
dataset = load_dataset("tweet_eval", "sentiment")
print(dataset)

In [ ]:
# Examine some examples from the dataset
for i in range(3):
    print(f"Text: {dataset['train'][i]['text']}")
    print(f"Label: {dataset['train'][i]['label']}")
    print("---")

In [ ]:
# Prepare the dataset for fine-tuning
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Load a small model for fine-tuning
model_name_ft = "distilbert-base-uncased"
tokenizer_ft = AutoTokenizer.from_pretrained(model_name_ft)
model_ft = AutoModelForSequenceClassification.from_pretrained(model_name_ft, num_labels=3)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare for training - use a small subset for demonstration
small_train_dataset = tokenized_datasets["train"].select(range(100))
small_eval_dataset = tokenized_datasets["validation"].select(range(50))

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    push_to_hub=False,  # Set to True if you want to upload to Hugging Face Hub
)

# Initialize Trainer
trainer = Trainer(
    model=model_ft,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
)

# Fine-tune the model (this will take a few minutes)
# If you're running this in a notebook with limited resources,
# you might want to skip this step or use even smaller datasets
trainer.train()

## 5. Saving and Sharing Your Model

After fine-tuning, you can save your model locally or push it to Hugging Face Hub to share with others.

In [ ]:
# Save the model and tokenizer locally
model_ft.save_pretrained("./my_finetuned_model")
tokenizer_ft.save_pretrained("./my_finetuned_model")

print("Model saved to ./my_finetuned_model")

In [ ]:
# If you want to push to the Hub (requires login)
# Uncomment and run these lines
"""
from huggingface_hub import HfFolder

# Define your model name (must be unique on your account)
model_name = "your-username/tweet-sentiment-model"

# Push the model to the Hub
if HfFolder.get_token() is not None:  # Check if logged in
    trainer.push_to_hub(model_name, private=True)  # Set private=False to make it public
    print(f"Model pushed to Hugging Face Hub: {model_name}")
else:
    print("You need to login first using login() function")
"""

## 6. Loading and Using Your Fine-tuned Model

Once you've saved your model, you can load it just like any other Hugging Face model.

In [ ]:
# Load the saved model from disk
loaded_tokenizer = AutoTokenizer.from_pretrained("./my_finetuned_model")
loaded_model = AutoModelForSequenceClassification.from_pretrained("./my_finetuned_model")

# Create a function to use the fine-tuned model
def analyze_tweet_sentiment(text):
    inputs = loaded_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = loaded_model(**inputs)
    
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    # For tweet_eval sentiment: 0 = negative, 1 = neutral, 2 = positive
    sentiment_map = {0: "negative", 1: "neutral", 2: "positive"}
    sentiment = sentiment_map[predicted_class]
    confidence = probabilities[0][predicted_class].item()
    
    return sentiment, confidence

In [ ]:
# Try the fine-tuned model with some examples
tweet_examples = [
    "Just had the best coffee ever! #happy",
    "Waiting in line at the DMV. So bored.",
    "Weather is cloudy today but I'm okay with it."
]

for text in tweet_examples:
    sentiment, confidence = analyze_tweet_sentiment(text)
    print(f"Text: {text}")
    print(f"Sentiment: {sentiment} (Confidence: {confidence:.4f})")
    print("---")

## 7. Advanced: Experimenting with Other Model Types

Hugging Face offers many different types of models for various tasks. Let's explore a few more examples.

### Named Entity Recognition

In [ ]:
from transformers import pipeline

# Create a named entity recognition pipeline
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

# Try it out
text = "My name is Sarah and I work at Google in New York City."
entities = ner_pipeline(text)

print("Named Entities:")
for entity in entities:
    print(f"{entity['word']}: {entity['entity']} (Score: {entity['score']:.4f})")

### Image Classification

In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image
import requests

# Download an example image
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# Display the image
image.show()  # This works in Jupyter notebook

# Load image classification model and feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

# Preprocess the image and get predictions
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits

# Get the predicted class
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

## 8. Deploying a Model for Production

In a production environment, you'll want to serve your model efficiently. There are several ways to deploy Hugging Face models:

### Option 1: Simple Flask API

Here's a basic example of creating a Flask API for your model:

In [ ]:
# Save this to a file called app.py
"""
from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

app = Flask(__name__)

# Load model (do this outside of route to load only once)
model_path = "./my_finetuned_model"  # Or use a model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json
    text = data['text']
    
    # Tokenize and predict
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    confidence = probabilities[0][predicted_class].item()
    
    # For tweet_eval sentiment: 0 = negative, 1 = neutral, 2 = positive
    sentiment_map = {0: "negative", 1: "neutral", 2: "positive"}
    sentiment = sentiment_map[predicted_class]
    
    return jsonify({
        'sentiment': sentiment,
        'confidence': confidence
    })

if __name__ == '__main__':
    app.run(debug=True)
"""

# To run this:
# python app.py
# Then, make POST requests to http://localhost:5000/predict with JSON data: {"text": "your text here"}

### Option 2: Using Hugging Face Inference API

If you push your model to Hugging Face Hub, you can use their Inference API:

In [ ]:
# Example code to use Hugging Face Inference API
"""
import requests

API_URL = "https://api-inference.huggingface.co/models/your-username/your-model-name"
headers = {"Authorization": f"Bearer {YOUR_API_TOKEN}"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()
    
output = query({"inputs": "The food was delicious and the service excellent."})
print(output)
"""

## 9. Best Practices and Tips

Here are some best practices when working with Hugging Face models:

1. **Model Selection**: Choose the smallest model that meets your needs to minimize resource usage and latency.

2. **Quantization**: For deployment, consider quantizing your model to reduce its size and increase inference speed.
   ```python
   # Example of quantizing a model
   from transformers import AutoModelForSequenceClassification
   import torch
   
   model = AutoModelForSequenceClassification.from_pretrained("your-model-name")
   quantized_model = torch.quantization.quantize_dynamic(
       model, {torch.nn.Linear}, dtype=torch.qint8
   )
   ```

3. **Batching**: Process multiple inputs in a batch to improve throughput.

4. **Caching**: Cache tokenization results and model outputs for repeated requests.

5. **Hardware Acceleration**: Use GPU or TPU when available for faster inference and training.

6. **Fine-tuning Strategies**: Consider techniques like parameter-efficient fine-tuning (e.g., LoRA, adapters) to reduce computational costs.

## 10. Exploring the Hugging Face Hub

The Hugging Face Hub contains thousands of models for different tasks. Here's how to search for models programmatically:

In [ ]:
from huggingface_hub import HfApi

api = HfApi()

# Search for models by task
models = api.list_models(filter="text-classification", limit=5)

print("Some text classification models on the Hub:")
for model in models:
    print(f"- {model.id} (Downloads: {model.downloads})")

## Conclusion

In this notebook, we've covered how to:
- Install and set up Hugging Face libraries
- Load pre-trained models from the Hub
- Use models for text classification, generation, and other tasks
- Fine-tune a model on a custom dataset
- Save and share your models
- Deploy models for production use

Hugging Face provides a powerful ecosystem that makes working with state-of-the-art models accessible to everyone. By combining pre-trained models with your own data, you can quickly develop powerful AI solutions for a wide range of tasks.

### Next Steps

To continue learning, you can:
1. Explore more model types on the [Hugging Face Hub](https://huggingface.co/models)
2. Read the [Transformers documentation](https://huggingface.co/docs/transformers/index)
3. Try fine-tuning on your own datasets
4. Experiment with advanced techniques like parameter-efficient fine-tuning

Happy modeling!